**Table of contents**<a id='toc0_'></a>    
- 1. [Problem 1: Optimal taxation with government consumption](#toc1_)    
- 2. [Problem 2: Labor adjustment costs](#toc2_)    
- 3. [Problem 3: Global optimizer with refined multi-start](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
#Importing and setting magics 

import numpy as np
from scipy import optimize
import scipy.optimize as opt
import sympy as sm
import matplotlib.pyplot as plt


plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

## 1. <a id='toc1_'></a>[Problem 1: Optimal taxation with government consumption](#toc0_)


Consider a worker choosing hours of labor, $L\in[0,24]$, to maximize utility: 

$$
\begin{align*}
V(w,\tau,G)&=\max_{L\in[0,24]}\ln\left(C^{\alpha}G^{1-\alpha}\right)-\nu\frac{L^{2}}{2}\\&\text{s.t.}\\&C=\kappa+(1-\tau)wL
\end{align*}
$$

where 

* $C$ is *private* consumption with weight $\alpha\in(0,1)$.
* $\kappa > 0$ is the *free private* consumption component.
* $(1-\tau)wL$ is the *costly private* consumption component.
* $w > 0 $ is the real wage.
* $\tau \in (0,1)$ is the labor-income tax rate.
* $G > 0 $ is *government* consumption with weight $1-\alpha$.
* $\nu > 0$ is the disutility of labor scaling factor


The baseline parameters are:

$$
\begin{align*}
\alpha &= 0.5\\
\kappa &= 1.0\\
\nu &= \frac{1}{2\cdot16^2} \\
w &= 1.0 \\ 
\tau &= 0.30 \\
\end{align*}
$$

# Question 1: 
**Verify that the optimal labor supply choice is $L^{\star}(\tilde{w}) =\frac{-\kappa+\sqrt{\kappa^{2}+4\frac{\alpha}{\nu}\tilde{w}^2}}{2\tilde{w}}$, where $\tilde{w} = (1-\tau)w$, for $G\in\left\{1.0 , 2.0\right\}$.**

I start by defining the optimal_labor_choice function. The function takes the parameters $\alpha$, $\kappa$, $\nu$, $\omega$, $\tau$ and G as inputs and calculates the optimal labor supply choice using the formula given in question 1. I define the baseline parameters as the given values and define the values of G as [1.0, 2.0]. 

I then add an verifucation step such that I can compare the results obtained from the code from the optimal_labor_choice function with the result of calculating the values "manually". Specifically, I use the np.isclose() function from the NumPy library to check if the two results are close to each other. In the end, the code prints wheter the reults match or not. If it matches, then I can confirm that the optimal labor supply choice is given by $L^{\star}(\tilde{w})$.


In [ ]:

def optimal_labor_choice(w, tau, alpha, kappa, nu, G):
    #Calculating the tilde_w value
    tilde_omega = (1 - tau) * w
    
    #Calculating the discriminant for the optimal labor supply choice formula
    discriminant = kappa**2 + 4 * alpha / nu * tilde_omega**2
    
    #Calculating the optimal labor supply choice using the formula
    L_star = (-kappa + np.sqrt(discriminant)) / (2 * tilde_omega)
    
    return L_star

#Defining baseline parameters
alpha = 0.5
kappa = 1.0
nu = 1 / (2 * 16**2)
omega = 1.0
tau = 0.30

#Defining the values G can take
G_values = [1.0, 2.0]

#Calculating the optimal labor supply choice for each G value
for G in G_values:
    #Calling the function to find the optimal labor supply choice
    L_star = optimal_labor_choice(omega, tau, alpha, kappa, nu, G)
    
    #Calculating the optimal labor supply choice MANUALLY using the formula
    calculated_L_star = (-kappa + np.sqrt(kappa**2 + 4 * alpha / nu * (1 - tau)**2 * omega**2)) / (2 * (1 - tau) * omega)
    
    print(f"For G = {G}:")
    print(f"Optimal labor supply choice from the code: L* = {L_star:.2f}")
    print(f"Optimal labor supply choice calculated manually: L* = {calculated_L_star:.2f}")
    
    #Verifying if the results match using isclose from np
    if np.isclose(L_star, calculated_L_star):
        print("Verification check: The results match!")
    else:
        print("Verification check: The results do not match")
    
    print()


# Question 2:
**Illustrate how $L^{\star}(\tilde{w})$ depends on $w$.**

I start by calculating the optimal labor supply choice for different values of omega. Then I plot the relationship using the plotting function from matplotlib. 

In [ ]:
#Generating a range of omega values
omega_values = np.linspace(0.01, 2.0, 100)

#Calculating the optimal labor supply choice for each omega value
L_star_values = [optimal_labor_choice(omega, tau, alpha, kappa, nu, G) for omega in omega_values]

#Plotting the relationship between omega and L_star
plt.plot(omega_values, L_star_values)
plt.xlabel('omega')
plt.ylabel('Optimal labor supply, L*')
plt.title('Fig 1: How the optimal labor supply depends on omega')
plt.grid(True)

#Showing the plot
plt.show()



**Interpretation:** 

The concave graph of the optimal labor supply, shows that there is diminishing marginal utility of consumption. That is people value their free time and prefer to have more leisure, which means that there is a labor-leisure trade-off. In oher words, there is a diminishing satisfaction from additional income as they work more.


We now consider a government, who chooses $\tau$ and spend all of the taxes on government consumption so:

$$
G = \tau w L^{\star}((1-\tau)w)
$$

# Question 3: 
**Plot the implied $L$, $G$ and worker utility for a grid of $\tau$-values.**


To calculate and plot the values of labor supply (L), government consumption (G), and worker utility based on a given set of parameters and a range of tax rates (tau), I start by defining three functions, where the first function "ootimal_labor_choice" is already used in the previous answers. 

The two new functions I define is "government_consumption" and "worker_utility". The government_consumption function calculates the government consumption based on the optimal labor supply. The worker_utility function calculates the worker's utility based on the optimal labor supply and consumption. 

To plot the implied $L$, $G$ and worker utility for a grid of $\tau$-values, I create three subplots. The first subplot shows how labor supply changes as the tax rate varies. The second subplot shows the corresponding government consumption for each tax rate. The third subplot shows the worker's utility as a function of the tax rate. I also adjust the figure size and use plt.tight_layout(), so that the figures do not collide with each oher.

In [ ]:
#Defining the three functions

#Defining the function for L
def optimal_labor_choice(omega, tau, alpha, kappa, nu):
    tilde_w = (1 - tau) * omega
    discriminant = kappa**2 + 4 * alpha / nu * tilde_w**2
    L_star = (-kappa + np.sqrt(discriminant)) / (2 * tilde_w)
    return L_star

#Defining the function for G
def government_consumption(omega, tau, alpha, kappa, nu):
    L_star = optimal_labor_choice(omega, tau, alpha, kappa, nu)
    return tau * omega * L_star

#Defining the function for worker_utility
def worker_utility(omega, tau, alpha, kappa, nu):
    L_star = optimal_labor_choice(omega, tau, alpha, kappa, nu)
    C = kappa + (1 - tau) * omega * L_star
    G = government_consumption(omega, tau, alpha, kappa, nu)
    return np.log(C**alpha * G**(1 - alpha)) - nu * L_star**2 / 2

#Defining a grid of tau values
tau_values = np.linspace(0.1, 0.9, 100)

#Calculating the implied values for L, G and worker utility referring to the tau_values linspace
L_values = [optimal_labor_choice(omega, tau, alpha, kappa, nu) for tau in tau_values]
G_values = [government_consumption(omega, tau, alpha, kappa, nu) for tau in tau_values]
utility_values = [worker_utility(omega, tau, alpha, kappa, nu) for tau in tau_values]

#Now plotting the results and adjusting the figure size so the subplots do not collide 
plt.figure(figsize=(10, 12))

#Fig 2: Labor Supply
plt.subplot(3, 1, 1)
plt.plot(tau_values, L_values)
plt.xlabel('τ')
plt.ylabel('L')
plt.title('Fig 2: The relationship between labor supply and tax rate')

#Fig 3: Government Consumption
plt.subplot(3, 1, 2)
plt.plot(tau_values, G_values)
plt.xlabel('τ')
plt.ylabel('G')
plt.title('Fig 3: The relationship between government consumption and tax rate')

#Fig 4: Worker Utility
plt.subplot(3, 1, 3)
plt.plot(tau_values, utility_values)
plt.xlabel('τ')
plt.ylabel('Worker Utility')
plt.title('Fig 4: The relationship between worker utility and tax rate')

#Ensuring the subplots do not overlap each other, so that they fit within the figure area
plt.tight_layout()

#Showing the plot
plt.show()


**Interpretation:**
1) Fig 2 shows that as the tax rate increases, the optimal labor supply decreases. This implies, that a higher tax rate discourages people to work. The cocae shape again represents a diminishing marginal utility of consumption and therefore a sign that the people prefers leisure time over work.

2) Fig 3 shows that as the tax rate increases, the government consumption increases. This is beacause an increase in the tax rate, will increase the tax revenue since the people will work less due to higher taxes. 

3) Fig 4 shows a concave graph, which implies a diminishing marginal utility of consumption and a increasing disutility of labor. In the beginning, as the tax rate increases, worker utility decreases. However, beyond a certain point, the marginal decrease in utility starts to diminish as the tax rate continues to increase. Therefore, there exists an optimal tax rate that maximizes worker utility that takes the trade-offs between consumption and leisure into consideration.

# Question 4:

**Find the socially optimal tax rate $\tau^{\star}\in(0,1)$ maximizing worker utility. Illustrate your result.**

First, I define a function called "social_optimal_tax_rate". This calculates the socially optimal tax rate by maximizing the negative worker utility function. Specifically, the opt.minimize function is used from the optimization library.

For each tax rate in tau_values, the code calculates the optimal labor supply using the optimal_labor_choice function.

Lastly, I plot the socially optimal tax rate in a figure that shows the relationship between the tax rate  and the labor supply like figure 2. The socially optimal tax rate is represented on the plot using plt.axvline that draws a vertical line at the optimal tax rate value.

In [ ]:
#Defining a function that calculates the socially optimal tax

def social_optimal_tax_rate(omega, alpha, kappa, nu):
    # Define the negative of the worker utility function
    def neg_worker_utility(tau):
        return -worker_utility(omega, tau, alpha, kappa, nu)

    #Maximizing the negative worker utility function
    result = opt.minimize(neg_worker_utility, x0=0.5, bounds=[(0, 1)])

    return result.x[0]

#Calculating the optimal labor supply for each tau_values
labor_values = [optimal_labor_choice(omega, tau, alpha, kappa, nu) for tau in tau_values]

#Plotting the labor supply as a function of the tax rate
plt.figure(figsize=(8, 6))
plt.plot(tau_values, labor_values)
plt.xlabel('Tax Rate')
plt.ylabel('Labor Supply')
plt.title('Fig 5: The relationship between labor supply and tax rate incl. the socially optimal tax rate')

#Calculating the socially optimal tax rate
social_opt_tau = social_optimal_tax_rate(omega, alpha, kappa, nu)

#Marking the socially optimal tax rate on the plot using axvline
plt.axvline(x=social_opt_tau, color='red', linestyle='--', label='Socially Optimal τ')
plt.legend()

#Printing the value of the socially optimal tax rate
print(f"The socially optimal tax rate is: {social_opt_tau:.2f}")

#Showing the plot
plt.show()


**Interpretation:**
Thus, the optimal tax rate that maximizes worker utility, which takes the trade-offs between consumption and leisure into consideration, is $\tau$ = 0.51. 

A more general preference formulation for the worker is:

$$
\begin{align*}
\mathcal{V}(w,\tau,G)&=\max_{L\in[0,24]}\frac{\left[ \left( \alpha C^{\frac{\sigma-1}{\sigma}}+(1-\alpha) G^{\frac{\sigma-1}{\sigma}} \right)^{\frac{\sigma}{\sigma-1} }\right]^{1-\rho}-1}{1-\rho}- \nu\frac{L^{1+\varepsilon}}{1+\varepsilon},\,\,\,\varepsilon,\rho,\sigma>0,\,\,\,\rho,\sigma\neq1\\&\text{s.t.}\\&C=\kappa+(1-\tau)wL
\end{align*}    
$$

Optimal labor supply is now $L^{\star}(\tilde{w},G)$.

Questions 5 and 6 must be answered with the general formulation, and for 2 different set of parameters:

- Set 1:  $\sigma = 1.001$, $\rho = 1.001$ and $\varepsilon = 1.0$.
- Set 2:  $\sigma = 1.5$, $\rho = 1.5$ and $\varepsilon = 1.0 $.

# Question 5:

**Find the $G$ that solves $G = \tau w L^{\star}((1-\tau)w,G)$ using the $\tau$ found in question 4.**

*Hint: First write code that solves the worker problem for given values of $G$ and $\tau$. Then find the correct G based on this.*

In [ ]:
def optimal_labor_choice(omega, tau, alpha, kappa, nu, sigma, rho, epsilon):
    tilde_w = omega
    discriminant = kappa**2 + 4 * alpha / nu * tilde_w**2
    L_star = (-kappa + np.sqrt(discriminant)) / (2 * tilde_w)
    return L_star

def target_equation(G, tau, omega, alpha, kappa, nu, sigma, rho, epsilon):
    L_star = optimal_labor_choice(omega, tau, alpha, kappa, nu, sigma, rho, epsilon)
    return G - tau * omega * L_star * (1 - tau) * omega


# Set 1: sigma = 1.001, rho = 1.001, epsilon = 1.0
sigma1 = 1.001
rho1 = 1.001
epsilon1 = 1.0

# Solve for G using the socially optimal tau value and parameter set 1
tau_optimal1 = social_opt_tau
G_solution1 = opt.root(target_equation, x0=0.5, args=(tau_optimal1, omega, alpha, kappa, nu, sigma1, rho1, epsilon1))
G_optimal1 = G_solution1.x[0]

# Set 2: sigma = 1.5, rho = 1.5, epsilon = 1.0
sigma2 = 1.5
rho2 = 1.5
epsilon2 = 1.0

# Solve for G using the socially optimal tau value and parameter set 2
tau_optimal2 = social_opt_tau
G_solution2 = opt.root(target_equation, x0=0.5, args=(tau_optimal2, omega, alpha, kappa, nu, sigma2, rho2, epsilon2))
G_optimal2 = G_solution2.x[0]

# Check if the calculated values of G satisfy the equation G = tau * omega * L_star * (1 - tau) * omega
equation_satisfied1 = np.isclose(G_optimal1, tau_optimal1 * omega * optimal_labor_choice(omega, tau_optimal1, alpha, kappa, nu, sigma1, rho1, epsilon1) * (1 - tau_optimal1) * omega)
equation_satisfied2 = np.isclose(G_optimal2, tau_optimal2 * omega * optimal_labor_choice(omega, tau_optimal2, alpha, kappa, nu, sigma2, rho2, epsilon2) * (1 - tau_optimal2) * omega)

# Print the results
print("Optimal G for Set 1:")
print(f"The value of G that satisfies the equation G = tau * omega * L_star * (1 - tau) * omega is: {G_optimal1:.2f}")
print(f"Equation satisfied: {equation_satisfied1}")

print("\nOptimal G for Set 2:")
print(f"The value of G that satisfies the equation G = tau * omega * L_star * (1 - tau) * omega is: {G_optimal2:.2f}")
print(f"Equation satisfied: {equation_satisfied2}")


**Question 6:** Find the socially optimal tax rate, $\tau^{\star}$, maximizing worker utility, while keeping $G = \tau w L^{\star}((1-\tau)w,G)$.

In [ ]:
# write your code here  

## 2. <a id='toc2_'></a>[Problem 2: Labor adjustment costs](#toc0_)

You own a hair salon. You employ hairdressers, $\ell_t$, to produce haircuts, $y_t = \ell_t$.

The wage for each haridresser is $w$.

The demand for haircuts implies that the price of haircuts you can charge is $p_t = \kappa_t y_t^{-\eta}$, where $\kappa_t$ is a demand-shock and $\eta \in (0,1)$ measures the elasticity of demand.

Profits are:

$$
\Pi_t = p_t y_t - w \ell_t = \kappa_t \ell_t^{1-\eta} - w \ell_t
$$

Baseline parameters are:
- $\eta = 0.5$
- $w = 1.0$

**Question 1:** Verify numerically that $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ maximises profits, for $\kappa\in\left\{1.0 , 2.0\right\}$.

In [ ]:
from hair_salon_model import HairSalonModel

model = HairSalonModel()


First, I create a HairSalonModel class, which is initialized with parameters eta (elasticity of demand) and w (wage for hairdressers).

I start by defining the values kappa can take and then calculate the optimal ell using the formula based on eta, kappa and w. Next, I calculate the profit for the optimal ell. 

To check if the optimal value for ell truly is the one that maximizes profits, I calculate if neighboring ell value results in a higher profit. Specifically, the code checks if the profit for the lower ell value is greater than the optimal profit, and if that is the case, it updates the optimal ell and profit. The same is done for the upper ell value. 

In [ ]:
kappa_values = [1.0, 2.0]
eta = model.eta  #Calling the eta from the model
w = model.w  #Calling the w from the model

for kappa in kappa_values:
    #Calculating the optimal ell using the formula
    optimal_ell = ((1 - eta) * kappa / w) ** (1 / eta)
    
    #Calculating the profit for the optimal ell
    optimal_profit = model.calculate_profit(kappa)
    
    #Calculating profits for neighboring values of ell to determine if the optimal ell value truly maximizes the profit or if the neighboring ell value results in a higher profit
    ell_lower = optimal_ell - 0.05  #Decreasing optimal_ell by 0.05
    ell_upper = optimal_ell + 0.05  #Increasing optimal_ell by 0.05
    
    #Calculating profits for the lower and upper ell values
    profit_lower = model.calculate_profit(kappa, ell_lower)
    profit_upper = model.calculate_profit(kappa, ell_upper)
    
    #Checking if the profit for the lower ell is greater than the optimal profit
    if profit_lower > optimal_profit:
        optimal_ell = ell_lower  # Set the lower ell as the new optimal ell
        optimal_profit = profit_lower  # Update the optimal profit
    
    #Checking if the profit for the upper ell is greater than the optimal profit
    if profit_upper > optimal_profit:
        optimal_ell = ell_upper  # Set the upper ell as the new optimal ell
        optimal_profit = profit_upper  # Update the optimal profit
    
    #Printing the result
    print(f"When kappa = {kappa}, the optimal ell value is: {optimal_ell}")
    print(f"The corresponding profit is: {optimal_profit}")



In [ ]:
# write your code here

We now consider a *dynamic* version of the model.

* The demand-shock is a so-called AR(1) in logs, 

$$
\log \kappa_{t} = \rho \log \kappa_{t-1} + \epsilon_{t},\,\,\, \epsilon_{t+1} \sim \mathcal{N}(-0.5\sigma_{\epsilon}^2,\sigma_{\epsilon})
$$

* Any hiring or firing implies a fixed adjustment cost, $\iota > 0 $.
* Future profits are discounted with a monthly factor of $R \in (0,1)$.

The initial demand shock is $\kappa_{-1} = 1$ and the planning horizon is 10 years, i.e. 120 months so $t \in \{0,1,2,\dots,119\}$. Initially you don't have any employees, $\ell_{-1}=0$


The *ex post* value of the salon is *conditional* on the shock series is:

$$
h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119}) = \left[\sum_{t=0}^{119}R^{-t}\left[\kappa_{t}\ell_{t}^{1-\eta}-w\ell_{t}-\boldsymbol{1}_{\ell_{t}\neq\ell_{t-1}}\iota\right]\right]
$$

The *ex ante* expected value of the salon can be approximated by

$$
H = \mathbb{E}[h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119})] \approx \frac{1}{K}\sum_{k=0}^{K} h(\epsilon_0^k,\epsilon_1^k,\dots,\epsilon_{119}^k)
$$

where each $k\in\{0,1,\dots,K-1\}$ is a random shock series. Maximizing profitability means maximizing $H$.


Baseline parameters are: 

- $\rho = 0.90$
- $\iota = 0.01$
- $\sigma_{\epsilon} = 0.10$
- $R = \left(1+0.01\right)^{1/12}$

**Question 2:** Calculate $H$ if the policy  $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ from question 1 is followed. Choose $K$ so the approximation is good enough to not affect your results substantially.

In [ ]:
# write your code here 

Next, we consider policies on the form:

$$

\ell_{t}=\begin{cases}
\ell_t^{\ast}  & \text{if }\left|\ell_{t-1}-\ell_t^{\ast} \right|>\Delta\\
\ell_{t-1} & \text{else }
\end{cases}
\\
\text{where}\,\,\ell_t^{\ast} = \left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}} \\

$$
With $\Delta \geq 0$ and $\Delta = 0$ being the previous policy.



**Question 3:** Calculate $H$ if the policy above was followed with $\Delta = 0.05$. Does it improve profitability?

In [ ]:
# write your code here

**Question 4:** Find the optimal $\Delta$ maximizing $H$. Illustrate your result.

In [ ]:
# write your code here


**Question 5:** Suggest an alternative policy you believe might improve profitability. Implement and test your policy.



In [ ]:
# write your code here

## 3. <a id='toc3_'></a>[Problem 3: Global optimizer with refined multi-start](#toc0_)

We consider the Griewank function:

$$ f(\boldsymbol{x}) = \sum^n_{i=1} \frac{x^2_i}{4000}-\prod^n_{i=1}\cos\left(\frac{x_i}{\sqrt{i}}\right)+1$$

The **global minimum** of this function is $f(0,0) = 0$ (remember: $\cos(0)=1$).<br>
But the function also have a lot of **local minima**.

In [ ]:
def griewank(x):
    return griewank_(x[0],x[1])
    
def griewank_(x1,x2):
    A = x1**2/4000 + x2**2/4000
    B = np.cos(x1/np.sqrt(1))*np.cos(x2/np.sqrt(2))
    return A-B+1

A **refined global optimizer with multi-start** is:

1. Choose *bounds* for $\mathbf{x}$ and *tolerance* $\tau > 0$.
2. Choose number of *warm-up iterations*, $\underline{K} > 0$ and *maximum number of iterations*, $K > \underline{K}$.
3. In each iteration for $k \in \{0,1,\dots,K-1\}$:

    A. Draw random $\mathbf{x}^k$ uniformly within chosen bounds.

    B. If $k < \underline{K}$ go to step E.

    C. Calculate $\chi^k = 0.50\cdot\frac{2}{1+\exp((k-\underline{K})/100)}$  

    D. Set $\mathbf{x}^{k0} = \chi^k \mathbf{x}^k + (1-\chi^k)\mathbf{x}^{\ast} $

    E. Run optimizer with $\mathbf{x}^{k0}$ as initial guess and $\mathbf{x}^{k\ast}$ as result.

    F. Set $\mathbf{x}^{\ast} = \mathbf{x}^{k\ast}$ if $k = 0$ or $f(\mathbf{x}^{k\ast}) < f(\mathbf{x}^{\ast})$

    G. If $f(\mathbf{x}^{\ast}) < \tau$ go to step 4.

4. Return the result $\mathbf{x}^{\ast}$.

As settings we choose:

* $x_1,x_2 \in  [-600,600]$
* $\tau = 10^{-8}$
* $\underline{K}=10$
* $K=1000$

The optimizer in Step 3.E is `BFGS` with a tolerance of $\tau$.

**Question 1:** Implement the refined global optimizer with multi-start. Illustrate how the effective initial guesses $\mathbf{x}^{k0}$ vary with the iteration counter $k$.

In [ ]:
# write your code here

**Question 2:** Is it a better idea to set $\underline{K} = 100$? Is the convergence faster?

In [ ]:
# write your code here